In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

# Outputs of the growth perturbation

In [3]:
ode_df_landscape = pd.read_csv('eem_growth_exp_landscape.csv').drop(['Unnamed: 0'], axis=1)
ode_df_timeseries = pd.read_csv('eem_growth_exp_timeseries.csv').drop(['Unnamed: 0'], axis=1)

In [27]:
melted_df

,time,magnitude,repro_changes,Variable,Value
0,0.000000,1,-0.030772,grasslandParkland,1.000000
1,0.340909,1,-0.030772,grasslandParkland,0.971721
2,0.681818,1,-0.030772,grasslandParkland,0.949198
3,1.022727,1,-0.030772,grasslandParkland,0.930618
4,1.363636,1,-0.030772,grasslandParkland,0.914342
...,...,...,...,...,...
4643995,515.375000,3,-0.247882,woodland,10.098363
4643996,515.750000,3,-0.247882,woodland,10.070039
4643997,516.000000,3,-0.247882,woodland,10.070039
4643998,516.375000,3,-0.247882,woodland,10.098363


In [31]:
melted_df

,time,magnitude,repro_changes,Variable,Value
0,0.000000,1,-0.030772,grasslandParkland,1.000000
1,0.340909,1,-0.030772,grasslandParkland,0.971721
2,0.681818,1,-0.030772,grasslandParkland,0.949198
3,1.022727,1,-0.030772,grasslandParkland,0.930618
4,1.363636,1,-0.030772,grasslandParkland,0.914342
...,...,...,...,...,...
6191995,515.375000,3,-0.247882,roeDeer,2.386499
6191996,515.750000,3,-0.247882,roeDeer,2.385824
6191997,516.000000,3,-0.247882,roeDeer,2.385824
6191998,516.375000,3,-0.247882,roeDeer,2.386499


In [ ]:
# adjust df for graphing
melted_df = ode_df_timeseries.melt(id_vars=['time', 'magnitude', 'repro_changes'],
                                   value_vars=['grasslandParkland', 'thornyScrub', 'woodland', 'roeDeer'],
                                   var_name='Variable', value_name='Value')

In [42]:
melted_df

,time,magnitude,repro_changes,Variable,Value
0,0.000000,1,-0.030772,grasslandParkland,1.000000
1,0.340909,1,-0.030772,grasslandParkland,0.971721
2,0.681818,1,-0.030772,grasslandParkland,0.949198
3,1.022727,1,-0.030772,grasslandParkland,0.930618
4,1.363636,1,-0.030772,grasslandParkland,0.914342
...,...,...,...,...,...
6191995,515.375000,3,-0.247882,roeDeer,2.386499
6191996,515.750000,3,-0.247882,roeDeer,2.385824
6191997,516.000000,3,-0.247882,roeDeer,2.385824
6191998,516.375000,3,-0.247882,roeDeer,2.386499


In [44]:
# Set style
sns.set_style("darkgrid")

# Create a colormap for repro_changes
repro_changes_cmap = sns.color_palette("cool", as_cmap=True)

# Create a FacetGrid scatterplot
g = sns.FacetGrid(melted_df, col='Variable', hue="repro_changes", col_wrap=2, height=4, aspect=1.2, sharey=False)

# Define the scatterplot for each subplot
g.map(sns.scatterplot, 'time', 'Value', palette=repro_changes_cmap)

# Set labels and title for each subplot
g.set_axis_labels('Time', 'Value')
g.set_titles('{col_name}')
g.fig.suptitle('Scatterplot of Value over Time by Variable', y=1.05)

# Add a colorbar
sm = plt.cm.ScalarMappable(cmap=repro_changes_cmap)
sm.set_array([])
g.fig.colorbar(sm, ax=g.axes)

# Adjust spacing between subplots
g.fig.tight_layout()

# Display the plot
plt.show()

TypeError: __init__() got an unexpected keyword argument 'style'